##Getting Started

Welcome! In this notebook we will analyze our uBiome data using qiime. 

Where you should be right now:
* ssh 'd into an EC2 instance of the qiime community AMI
* Have copied the processed uBiome fastq files onto the EC2 instance
* These files should be in a folder in the home directory called ubiome_data, and should be labelled with a unique sequence ID followed by \_R1_.fastq and \_R2\_.fastq

## Creating a mapping file

You also need to create a mapping file for your samples. The qiime mapping file contains information about the sample IDs, barcodes, primers, and information about the samples that you chose to include (for example, sampling location, date, and species). These files must be in a very particular format, read about it [here](http://qiime.org/documentation/file_formats.html). For an example file, look [here](https://docs.google.com/spreadsheets/u/1/d/1FXHtTmvw1gM4oUMbRdwQIEOZJlhFGeMNUvZmuEFqpps/pubhtml?gid=0&single=true).

Ubiome data is slightly different than most data that is inputted into qiime because it has already been demultiplexed and the primers trimmed, which means those two columns will remain blank in your mapping file. However, they still must be present. 

If you have not already, generate a mapping file for your samples. If it is easiest, [download this example](https://raw.githubusercontent.com/oddaud/ubiome_tutorials/master/map.txt) and edit the data in excel. This is most easily done in an excel spreadsheet, but the file must be saved as a tab seperated values (.tsv). Transfer this file to the AWS community server, and place it in the ubiomeSamples folder. 

In [ ]:
##DO NOT evaluate this cell! This command must be edited, copied, and run from yout LOCAL terminal. 
scp -i [location of your key] [path to your map.tsv] ubuntu@[public DNS]:/home/ubuntu/ubiomeSamples/

You can now test to see if your mapping file is formated correctly. 

In [1]:
from functools import partial
from os import chdir
from IPython.display import FileLinks, FileLink

In [2]:
#enter ubiomeSamples directory
chdir('ubiomeSamples')

In [3]:
!pwd

/home/ubuntu/ubiomeSamples


In [27]:
!validate_mapping_file.py -o vmf-map/ -m ../map.tsv

Errors and/or warnings detected in mapping file.  Please check the log and html file for details.


In [30]:
FileLinks('vmf-map-bad/')

Path ( vmf-map-bad ) doesn't exist. It may still be in the process of being generated, or you may have the incorrect path.

##Merging paired end reads

 Next, we will merge paired end reads using the qiime script [multiple_join_paired_ends.py](http://qiime.org/scripts/multiple_join_paired_ends.html). To view the options for any qiime script (and most scripts in general), use the -h option:

In [40]:
!multiple_join_paired_ends.py -h

The V4 region is 254 bp generally, but may deviate by a few bp in some cases. If you examine the length of each read in either the R1 or R2 file, you will notice that they are each 150 bp long. With a little math, you will notice then that each V4 region is covered completely by a read from either end, with an overlap of 46 bp in the middle.

To merge our paired end reads, specify inputs, outputs, and the parameters. The parameters must be passed in a seperate file because multiple_join_paired_ends.py calls a different function, join_paired_ends.py, and we need to use parameters other than the default for join_paired_ends.py. To create a params file, we will write a quick file with python:

In [45]:
with open('join_paired_end_reads_params.txt', 'w+') as f:
                        f.write('join_paired_ends:perc_max_diff    99')

Now to join the paired ends. Don't forget to wait until the process is finishes ([\*] is gone) to move on!

In [46]:
!multiple_join_paired_ends.py -i ./ -o merged -p join_paired_endreads_params.txt

At this point you have a file called \*merged.fastq for each sample. Qiime requires one single fastq file with each read labelled with a sample ID to do future analyses. Time to merge again. 

This requires some work by hand. To begin, write out each file in alphabetical order, separated by commas but NO SPACES. You can use ls -l to see them. 

In [10]:
chdir('merged')

/home/ubuntu/ubiomeSamples/merged


In [11]:
!pwd
!ls -l

/home/ubuntu/ubiomeSamples/merged
total 462360
-rw-rw-r-- 1 ubuntu ubuntu      1886 Feb 26 22:13 log_20170226221319.txt
-rw-rw-r-- 1 ubuntu ubuntu  82620538 Feb 26 22:13 ssr_8730_R1_.merged.fastq
-rw-rw-r-- 1 ubuntu ubuntu 204009871 Feb 26 22:13 ssr_8737_R1_.merged.fastq
-rw-rw-r-- 1 ubuntu ubuntu 186813427 Feb 26 22:13 ssr_8797_R1_.merged.fastq


For example, I might write: 

XXXXXXX

Next, write down corresponding sample numbers, also separated by commas. MAKE SURE THEY CORRESPOND EXACTLY. For example:

AB002,AB003,AB004

The script we are going to use is called split_libraries_fastq.py. This script performs a number of functions, including demultiplexing of samples and quality filtering. Because fastq files provided by uBiome are already demultiplexed, we must pass this script the sample name associated with each file. For more information on qiime quality filtering, [check out this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3531572/). 

In [ ]:
#EDIT THIS CODE BEFORE YOU EVALUATE IT!
!split_libraries_fastq.py -i [your list of fastq files] --sample_ids [your list of sample IDs] -o slout --barcode_type 'not-barcoded' -v

In [14]:
!split_libraries_fastq.py -i ssr_8730_R1_.merged.fastq,ssr_8737_R1_.merged.fastq,ssr_8797_R1_.merged.fastq --sample_ids AB002,AB003,AB004 -o out --barcode_type 'not-barcoded' -v

Now you (hopefully) have one file with all your mereged, cleaned, and filtered fastq reads in it. 
These reads should be identified with their sample numbers, for example:

\>AB002_0 NB501532:61:HGWJVAFXX:1:11101:9255:1102 1:N:0:CTACTCGA+CACATACG orig_bc=AAAAAAAAAAAA new_bc=AAAAAAAAAAAA bc_diffs=0
CGTGTGCCAGCAGCCGCGGTAAGACGGGGGGGGCAAGTGTTCTTCGGAATGACTGGGCGTAAAGGGCACGTAGGCGGTGAATCGGGTTGAAAGTGAAAGTCGCCAAAAACTGGTGGAATGCTCTCGAAACCAATTCACTTGAGTGAGACAGTGGAATTTCGTGTGTAGGGGTGAAATCCGGAGATCTACGAAGGAAGGCCAAAAGCGAAGGCAGCTCTCTGGGTCCCCACCGACGCTGGAGTGCGAAAGCATGGGGAGCGAACGGGATTAGAAACCCCAGTAGTCCGGT
\>AB002_1 NB501532:61:HGWJVAFXX:1:11101:5822:1106 1:N:0:CTACTCGA+CACATACG orig_bc=AAAAAAAAAAAA new_bc=AAAAAAAAAAAA bc_diffs=0
CGTGTGCCAGCAGCCGCGGTAATACAGAGGATGCAAGCGTTATCCGGAATGATTGGGCGTAAAGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCGGTGGAAACTACCAAGCTGGAGTACGGTCAGAGGGAATTTCCGGTGGAGCGGTGAAATGCGTAGAGATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAGGGGAGCGAATGGGATTAGAAACCCCAGTAGTCCGGA


##OTU Picking

Next up, we'll be picking otu's! We will be using the script [pick_open_reference_otus.py](http://qiime.org/scripts/pick_open_reference_otus.html). We are specifying one important parameter, here, setting enable_rev_strand_match to True. This allows sequences to be considered a match if either their forward or reverse sequence hits the database. To set this parameter, we need to create a params file again. 

In [15]:
with open('uc_fast_params.txt', 'w+') as f:
                        f.write('pick_otus:enable_rev_strand_match True')

pick_open_reference_otus.py can take a looooong time depending on how big seqs.fna is. Around 3 hours for 15 merged samples, 1 hour for 3 samples. 

In [17]:
!pick_open_reference_otus.py -o otus/ -i slout/seqs.fna -p uc_fast_params.txt

DESCRIPTION COPIED FROM [Illumina Overview Tutorial](http://nbviewer.jupyter.org/github/biocore/qiime/blob/1.9.1/examples/ipynb/illumina_overview_tutorial.ipynb)

The primary output that we get from this command is the *OTU table*, or the number of times each operational taxonomic unit (OTU) is observed in each sample. QIIME uses the Genomics Standards Consortium Biological Observation Matrix standard (BIOM) format for representing OTU tables. You can find additional information on the BIOM format [here](http://www.biom-format.org), and information on converting these files to tab-separated text that can be viewed in spreadsheet programs [here](http://biom-format.org/documentation/biom_conversion.html). Several OTU tables are generated by this command. The one we typically want to work with is ``otus/otu_table_mc2_w_tax_no_pynast_failures.biom``. This has singleton OTUs (or OTUs with a total count of 1) removed, as well as OTUs whose representative (i.e., centroid) sequence couldn't be aligned with [PyNAST](http://bioinformatics.oxfordjournals.org/content/26/2/266.long). It also contains taxonomic assignments for each OTU as *observation metadata*.

The open-reference OTU picking command also produces a phylogenetic tree where the tips are the OTUs. The file containing the tree is ``otus/rep_set.tre``, and is the file that should be used with ``otus/otu_table_mc2_w_tax_no_pynast_failures.biom`` in downstream phylogenetic diversity calculations. The tree is stored in the widely used [newick format](http://scikit-bio.org/docs/latest/generated/skbio.io.newick.html).

To view the output of this command, call ``FileLink`` on the ``index.html`` file in the output directory.

In [21]:
FileLink('otus/index.html')

/home/ubuntu/ubiomeSamples/merged/otus/index.html

We can calculate some statistics on our OTUs using the biom program. 

In [24]:
!biom summarize-table -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom

Num samples: 3
Num observations: 13059
Total count: 627933
Table density (fraction of non-zero values): 0.378

Counts/sample summary:
 Min: 103333.0
 Max: 279534.0
 Median: 245066.000
 Mean: 209311.000
 Std. dev.: 76247.462
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
 AB002: 103333.0
 AB004: 245066.0
 AB003: 279534.0


Now we have to pick what sequencing depth we would like to perform our rarefacation analysis on. Because different samples have different sequencing depths (i.e. numbers of reads) it would be wrong to assume that all had sequenced the diversity of the sample to the same extent. One common way to normalize sequencing depth across samples is to choose the sample with the lowest depth, and discard reads randomly in the remaining samples so that they all have the depth of the lowest sample. For a more detailed discussion on this, [see here](http://www.wernerlab.org/teaching/qiime/overview/e).

Looking at the summary of your .biom file, note down the min number of reads in your samples (Min under Counts/sample summary). This is your sequencing depth. If you have one file that is far lower than all the others, it might make more sense to discard that sample from future analyses. You have to decide which is better: to discard one sample, or discard all the data from the other samples to acheive the same low sequencing depth as that one sample. 

##Core Diversity Analysis

The script core_diversity_analyses.py will give us a first glance look at some important information about our samples. At this point, we need our mapping file. 

-e specifies the sequencing depth you will be using. The one I have is based on my data, replace it with something appropriate for yours! This step can also take a long time (~30 minutes)


/home/ubuntu/ubiomeSamples/merged


In [33]:
!core_diversity_analyses.py -o cdout/ -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom -m ../map.tsv -t otus/rep_set.tre -e 103333

Traceback (most recent call last):
  File "/usr/local/bin/core_diversity_analyses.py", line 202, in <module>
    main()
  File "/usr/local/bin/core_diversity_analyses.py", line 199, in main
    status_update_callback=status_update_callback)
  File "/usr/local/lib/python2.7/dist-packages/qiime/workflow/core_diversity_analyses.py", line 252, in run_core_diversity_analyses
    status_update_callback=status_update_callback)
  File "/usr/local/lib/python2.7/dist-packages/qiime/workflow/downstream.py", line 183, in run_beta_diversity_through_plots
    close_logger_on_success=close_logger_on_success)
  File "/usr/local/lib/python2.7/dist-packages/qiime/workflow/util.py", line 122, in call_commands_serially
    raise WorkflowError(msg)
qiime.workflow.util.WorkflowError: 

*** ERROR RAISED DURING STEP: Make emperor plots, weighted_unifrac)
Command run was:
 make_emperor.py -i cdout//bdiv_even103333//weighted_unifrac_pc.txt -o cdout//bdiv_even103333//weighted_unifrac_emperor_pcoa_plot/ -m ../map

In [23]:
FileLink('cdout/index.html')

/home/ubuntu/ubiomeSamples/merged


Great! You are done. You can look through the outputs using the file link above. 

Next, we need to transfer these outputs back to our local computer, for analysis with RDP.

To transfer files, use scp from a local terminal window.


In [ ]:
scp -r -i [location of your key] ubuntu@[public DNS]:/home/ubuntu/ubiomeSamples/merged/ [location you want to save the files to]

Great! You are all done with the community AMI. You may stop it, or terminate it. 